# Example Notebook

This notebook will load some sample data from wind and create a compressibility scalogram.

In [2]:
# install some packages
# !pip install pyspedas
# !pip install pandas
# !pip install matplotlib

# install the Time-Series-Scalogram package
# !pip install Time-Series-Scalogram

In [19]:
# load the ts_scalogram module
from Time_Series_Scalogram import ts_scalogram

# load pyspedas
import pyspedas
from pytplot import tplot, get_data, time_string

# load pandas
import pandas as pd

# load numpy
import numpy as np

## Download Data

In [20]:
# download some wind data
# case 375 from https://wind.nasa.gov/ICME_catalog/ICME_catalog_viewer.php

t0 = pd.Timestamp('2018-06-06 11:23:00')
t1 = pd.Timestamp('2018-06-07 08:29:00')
dt = t1 - t0

# load data from a larger window
t00 = t0 - dt
t11 = t1 + dt


mfi_vars = pyspedas.wind.mfi(trange=[t00.strftime('%Y-%m-%d %H:%M:%S'), t11.strftime('%Y-%m-%d %H:%M:%S')], datatype='h4-rtn')
mag_mfi = get_data('BRTN')
index = [pd.Timestamp(x) for x in time_string(mag_mfi.times)]
df_mfi = pd.DataFrame(mag_mfi.y, index=index, columns= ['Bx', 'By', 'Bz'])
df_mfi['Btot'] = np.sqrt(df_mfi['Bx']**2 + df_mfi['By']**2 + df_mfi['Bz']**2)
df_mfi.sort_index(inplace=True)

03-Nov-24 21:59:22: File is current: wind_data/wind_masters/wi_h4-rtn_mfi_00000000_v01.cdf
03-Nov-24 21:59:22: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/wind/mfi/mfi_h4-rtn/2018/
03-Nov-24 21:59:23: File is current: wind_data/mfi/mfi_h4-rtn/2018/wi_h4-rtn_mfi_20180605_v05.cdf
03-Nov-24 21:59:23: File is current: wind_data/mfi/mfi_h4-rtn/2018/wi_h4-rtn_mfi_20180606_v05.cdf
03-Nov-24 21:59:24: File is current: wind_data/mfi/mfi_h4-rtn/2018/wi_h4-rtn_mfi_20180607_v05.cdf
03-Nov-24 21:59:24: File is current: wind_data/mfi/mfi_h4-rtn/2018/wi_h4-rtn_mfi_20180608_v05.cdf


## Process Data

In [21]:
# smooth |B|
Btot = df_mfi['Btot'] # load magnetic magnitude
Btot_smooth = Btot.apply(np.log10).rolling('5min').median().apply(lambda x: 10**x) # smooth the data with 5 minute log median
Btot_normed = Btot/Btot_smooth # normalize the time series

# calculate the windows
upper_bound = ((index[-1] - index[0])/2).ceil('1H') # upper bound is half of the time range
t_step = np.timedelta64(5, 'm') # a suitable time step
yrange = [np.timedelta64(0,'m'), upper_bound] # range of the y axis
num_wins = 10000
win_step = t_step*2
win0 = t_step * 2 # should be nX of t_step, where n is an integer, here I use 2
wins = np.array(np.arange(num_wins)*win_step+win0, dtype = np.timedelta64) # list of "raw" windows
ind = (wins >= yrange[0]) & (wins <= yrange[1]) # select the windows within the yrange
wins = wins[ind] # final windows

03-Nov-24 22:05:12: /var/folders/g1/jbj1s_4x7wn1f67ffjt2_thm0000gn/T/ipykernel_75410/2715919445.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  upper_bound = ((index[-1] - index[0])/2).ceil('1H') # upper bound is half of the time range



## Calculate the Scalogram

In [22]:
savpath = "./temp/"
scans, settings = ts_scalogram.TimeSeriesScalogram(
    Btot_normed,
    wins,
    t_step,
    chunksize = 500,
    Ncores = 16,
    savpath = savpath
)

ValueError: Please specify the absolute path of pixel_func.py